In [3]:

from google.cloud import bigquery

client = bigquery.Client(project="prod-organize-arizon-4e1c0a83")

query = """
WITH rv AS (

SELECT DISTINCT
d.countyname
,CASE
    WHEN p.partyaffiliation = 'DEM' THEN 'DEM'
    WHEN p.partyaffiliation = 'REP' THEN 'GOP'
    ELSE '3RD'
  END AS party
, COUNT(DISTINCT p.dwid) AS RV

FROM `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__person` AS p

INNER JOIN  `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON p.dwid = d.dwid

WHERE d.state = 'AZ'
AND p.state = 'AZ'
AND p.voterstatus = 'active'
AND d.statehousedistrict = '19'

GROUP BY 1,2
ORDER BY 1
)

, lv AS(

SELECT DISTINCT
d.countyname
,CASE
    WHEN p.partyaffiliation = 'DEM' THEN 'DEM'
    WHEN p.partyaffiliation = 'REP' THEN 'GOP'
    ELSE '3RD'
  END AS party
, COUNT(DISTINCT p.dwid) AS LV

FROM `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__person` AS p

INNER JOIN  `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON p.dwid = d.dwid

LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__models` AS m
  ON d.dwid = m.dwid

WHERE d.state = 'AZ'
AND p.state = 'AZ'
AND p.voterstatus = 'active'
AND d.statehousedistrict = '19'
AND m.catalistmodel_voteprop2026 >= 70

GROUP BY 1,2
ORDER BY 1
)

SELECT DISTINCT
r.countyname
, r.party
, r.rv
, l.lv
FROM rv AS r

LEFT JOIN lv AS l
  ON r.party = l.party
  AND r.countyname = l.countyname

ORDER BY 1,2
"""

df = client.query(query).to_dataframe()
df.head()

,countyname,party,rv,lv
0,COCHISE,3RD,27815,8299
1,COCHISE,DEM,16024,8567
2,COCHISE,GOP,31187,18736
3,GRAHAM,3RD,5081,1198
4,GRAHAM,DEM,3042,1571


In [4]:
%pip install -q db-dtypes pyarrow

In [11]:

import altair as alt
import pandas as pd

party_colors = {
    "DEM": "#1f77b4",   # Dem blue
    "GOP": "#d62728",   # GOP red
    "3RD": "#7f7f7f"    # neutral gray
}

alt.renderers.enable("default")

RendererRegistry.enable('default')

In [12]:
rv_stack = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        y=alt.Y("countyname:N", sort="-x", title="County"),
        x=alt.X("rv:Q", title="Registered Voters (RV)"),
        color=alt.Color(
            "party:N",
            title="Party",
            scale=alt.Scale(
                domain=list(party_colors.keys()),
                range=list(party_colors.values()),
            ),
        ),
        tooltip=["countyname", "party", "rv", "lv"],
    )
    .transform_filter("datum.rv > 0")
)

rv_stack


alt.Chart(...)

In [13]:
rv_stack = rv_stack.interactive()
rv_stack.save("rv_stack.html")
